In [1]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ultralytics import YOLO

In [5]:
#best_model = YOLO("best_model.pt")

#print(f"The task type of the model is: {best_model.task}")

In [2]:
TASK2METRIC = {
    "detect": "metrics/mAP50(B)", #detect 지표를 mAP50-95에서 50으로 변경하기 위함
    "segment": "metrics/mAP50-95(M)",
    "classify": "metrics/accuracy_top1",
    "pose": "metrics/mAP50-95(P)",
    "obb": "metrics/mAP50-95(B)",
}

In [5]:
experiment_name = 'tune'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# Load a YOLOv8n model
best_model_tuning = YOLO("yolov8n.pt")

task = best_model_tuning.task
metric = TASK2METRIC[task] 

result_grid = best_model_tuning.tune(
    data="/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml",
    epochs=200,
    use_ray=True,
    name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
    gpu_per_trial=1  # 각 트라이얼에 1개의 GPU 할당
)

(_tune pid=38164) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=38162) engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False

  0%|          | 0.00/6.25M [00:00<?, ?B/s]
  4%|▎         | 224k/6.25M [00:00<00:02, 2.12MB/s]
 25%|██▍       | 1.54M/6.25M [00:00<00:00, 5.21MB/s]
 95%|█████████▌| 5.94M/6.25M [00:01<00:00, 3.23MB/s]
100%|██████████| 6.25M/6.25M [00:01<00:00, 3.99MB/s]
100%|██████████| 6.25M/6.25M [00:01<00:00, 3.82MB/s]
100%|██████████| 6.25M/6.25M [00:02<00:00, 2.99MB/s]


(_tune pid=38162) AMP: checks passed ✅
(_tune pid=38163) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB) [repeated 3x across cluster]
(_tune pid=38163) engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes

train: Scanning /home/under1/Detect/dataset/sagittal/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
  0%|          | 0.00/6.25M [00:00<?, ?B/s] [repeated 3x across cluster]
100%|██████████| 6.25M/6.25M [00:02<00:00, 3.01MB/s] [repeated 62x across cluster]


(_tune pid=38161) Plotting labels to runs/detect/tune/tune_raytune/labels.jpg... 
(_tune pid=38164) AMP: checks passed ✅ [repeated 3x across cluster]
(_tune pid=38162) optimizer: 'optimizer=auto' found, ignoring 'lr0=0.046883883136965905' and 'momentum=0.6323953957993099' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
(_tune pid=38162) optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0003323633793075994), 63 bias(decay=0.0)
(_tune pid=38162) Image sizes 640 train, 640 val
(_tune pid=38162) Using 8 dataloader workers
(_tune pid=38162) Logging results to runs/detect/tune/tune_raytune
(_tune pid=38162) Starting training for 200 epochs...
(_tune pid=38162) 
(_tune pid=38162)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    200/200       2.1G   0.009329      1.046      1.195          6        640:   0%|          | 0/14 [00:00<?, ?it/s]
    199/200       2.1G   0.009564      1.106      1.199         10        640:  86%|████████▌ | 12/14 [00:00<00:00, 12.46it/s] [repeated 8x across cluster]
    200/200       2.1G     0.0099      1.115      1.205         11        640:  14%|█▍        | 2/14 [00:00<00:00, 12.13it/s]
    200/200       2.1G     0.0094       1.09      1.173         10        640:  29%|██▊       | 4/14 [00:00<00:00, 13.62it/s] [repeated 11x across cluster]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:00<00:00,  8.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  9.71it/s]


(_tune pid=44553) 
(_tune pid=44553) 200 epochs completed in 0.114 hours.
(_tune pid=44553) Optimizer stripped from runs/detect/tune/tune_raytune/weights/last.pt, 6.3MB
(_tune pid=44553) Optimizer stripped from runs/detect/tune/tune_raytune/weights/best.pt, 6.3MB
(_tune pid=44553) 
(_tune pid=44553) Validating runs/detect/tune/tune_raytune/weights/best.pt...
(_tune pid=44553) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=44553) Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]
    200/200       2.1G   0.008957      1.071      1.158         11        640:  71%|███████▏  | 10/14 [00:00<00:00, 12.36it/s] [repeated 7x across cluster]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:00<00:00,  9.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.67it/s]


(_tune pid=44553)               negative         30         30      0.539      0.967      0.783      0.464
(_tune pid=44553)               positive         22         24      0.545      0.458      0.573      0.319
(_tune pid=44553) Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.6ms postprocess per image
(_tune pid=44553) Results saved to runs/detect/tune/tune_raytune
(_tune pid=44553)                    all         52         54      0.542      0.713      0.678      0.391 [repeated 2x across cluster]
(_tune pid=44553)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


2024-07-03 11:01:24,154	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/under1/Detect/jeongui/sagittal/runs/detect/tune2/_tune_2024-07-03_10-49-05' in 0.0170s.
2024-07-03 11:01:24,173	INFO tune.py:1041 -- Total run time: 738.26 seconds (738.21 seconds for the tuning loop).


In [6]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

Trial #0: Configuration: {'lr0': 0.056643642485429135, 'lrf': 0.04776314042577309, 'momentum': 0.7572215642743199, 'weight_decay': 0.0004551069335432381, 'warmup_epochs': 1.9593949519225073, 'warmup_momentum': 0.5642242838724062, 'box': 0.16091465550228212, 'cls': 0.994999036511786, 'hsv_h': 0.044042753267944636, 'hsv_s': 0.6423159125919778, 'hsv_v': 0.12591175152114356, 'degrees': 3.9656712154694667, 'translate': 0.8451038180601376, 'scale': 0.46008764643481553, 'shear': 8.574277725751742, 'perspective': 0.0005498159553405502, 'flipud': 0.5134370487447365, 'fliplr': 0.7991916606016591, 'bgr': 0.2797545098604919, 'mosaic': 0.7914118884049222, 'mixup': 0.46091362944209646, 'copy_paste': 0.5484175033536863, 'data': '/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}, Last Reported Metrics: {'metrics/precision(B)': 0.43699, 'metrics/recall(B)': 0.64167, 'metrics/mAP50(B)': 0.43797, 'metrics/mAP50-95(B)': 0.23184, 'val/box_loss': 

In [7]:
# Extract the best trial configuration
best_result = result_grid.get_best_result(metric=metric, mode="max")
best_config = best_result.config
print("Best hyperparameters found were: ", best_config)

Best hyperparameters found were:  {'lr0': 0.09100706546466665, 'lrf': 0.5347844212147748, 'momentum': 0.6148749897228741, 'weight_decay': 0.0003741530390944526, 'warmup_epochs': 1.699395107083082, 'warmup_momentum': 0.819821603765741, 'box': 0.06279108325557228, 'cls': 0.66757227313073, 'hsv_h': 0.010609909296838494, 'hsv_s': 0.8854570634017006, 'hsv_v': 0.24994542237708187, 'degrees': 10.99879888869428, 'translate': 0.6922416792593282, 'scale': 0.3253418492086288, 'shear': 2.052873531621403, 'perspective': 0.0008426481164459991, 'flipud': 0.14537447687533733, 'fliplr': 0.977580247292482, 'bgr': 0.7772407951059513, 'mosaic': 0.6778095671936483, 'mixup': 0.43827584365436556, 'copy_paste': 0.6344088746540981, 'data': '/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}


In [9]:
experiment_name = 'train'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# 'data' 항목 제거
if 'data' in best_config:
    del best_config['data']
if 'name' in best_config:
    del best_config['name']

# 최고의 하이퍼파라미터로 모델 재훈련
best_model = YOLO("yolov8n.pt")  # 모델 재초기화
best_results = best_model.train(data="sagittal_t1wce_2_class.yaml",
                                optimizer='SGD',
                                augment=True,
                                name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
                                **best_config)

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train_raytune, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show

train: Scanning /home/under1/Detect/dataset/sagittal/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/train_raytune/labels.jpg... 
optimizer: SGD(lr=0.09100706546466665, momentum=0.6148749897228741) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0003741530390944526), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train/train_raytune
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      2.21G    0.01486       6.21      1.648          4        640: 100%|██████████| 14/14 [00:02<00:00,  5.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.37it/s]

                   all         52         54   4.96e-05     0.0167   2.66e-05   5.31e-06


...

    200/200      2.13G    0.01048      1.573      1.297          1        640: 100%|██████████| 14/14 [00:01<00:00, 11.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  9.90it/s]

                   all         52         54      0.474      0.829      0.563      0.355



200 epochs completed in 0.104 hours.
Optimizer stripped from runs/detect/train/train_raytune/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/train_raytune/weights/best.pt, 6.3MB

Validating runs/detect/train/train_raytune/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.23it/s]


                   all         52         54      0.467      0.778       0.57       0.39
              negative         30         30      0.584       0.89      0.739       0.52
              positive         22         24      0.349      0.667      0.402      0.261
Speed: 0.1ms preprocess, 14.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train/train_raytune


In [10]:
best_results.results_dict

{'metrics/precision(B)': 0.46657260231674946,
 'metrics/recall(B)': 0.7781156156156156,
 'metrics/mAP50(B)': 0.5702838400966688,
 'metrics/mAP50-95(B)': 0.39015896849149745,
 'fitness': 0.40817145565201457}

In [11]:
experiment_name = 'val'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model = YOLO("/home/under1/Detect/jeongui/sagittal/runs/detect/train/train_raytune/weights/best.pt")
best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'))

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.46it/s]


                   all         52         54      0.488      0.804      0.559       0.38
              negative         30         30      0.628      0.899      0.739       0.51
              positive         22         24      0.349      0.708      0.379       0.25
Speed: 0.2ms preprocess, 9.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val/val_raytune


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ef17cd51390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [12]:
# test

experiment_name = 'test'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="test")

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


val: Scanning /home/under1/Detect/dataset/sagittal/labels/test.cache... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  4.18it/s]


                   all         70         77      0.468      0.744      0.494      0.309
              negative         35         39      0.482      0.718      0.453      0.312
              positive         35         38      0.455       0.77      0.536      0.307
Speed: 0.2ms preprocess, 5.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/test/test_raytune


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ef1940b0d00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804